In [ ]:
from datetime import datetime
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(project_root)

from ultralytics import YOLO, settings
import mlflow

from custom_dataset.data_preparation import YOLODataProcessor

In [ ]:
# Set mlflow experiment

experiment_name = 'yolo_train'

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(experiment_name)

In [ ]:
# Get all runs

experiment = mlflow.get_experiment_by_name(experiment_name)
df_runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
df_runs.empty

# 1. Dataset prepatation

In [ ]:
dataset_name = 'yolo-ocr-dataset_yolo'
dataset_path = os.path.join(project_root, 'custom_dataset', 'data', dataset_name)
dataset_config = f'{dataset_name}.yaml'

data_processor = YOLODataProcessor(dataset_path)
data_processor(
    config_path=dataset_config,
    train_frac=0.95,
    val_frac=0.025,
    restructure=False,
)

# 2. Train

In [ ]:
# Give name to current run (model) and create it
v = 1
date = datetime.now().strftime("%d.%m.%y")
model_name = f'{date}_{dataset_name}_v{v}'
if not df_runs.empty:
    while (df_runs['tags.mlflow.runName'] == model_name).any():
        v += 1
        model_name = f'{date}_{dataset_name}_v{v}'
print(model_name)

with mlflow.start_run(run_name=model_name) as run:
    run_id = run.info.run_id
os.environ['MLFLOW_RUN_ID'] = run_id
print(run_id)

In [ ]:
# Store run_name and run_id for downstream use

%store run_id
%store model_name

In [ ]:
model = YOLO("yolo11n.pt")
# print(settings)

# All params here https://docs.ultralytics.com/modes/train/#train-settings
model.train(
    data=f'{dataset_name}.yaml',
    epochs=3,
    imgsz=224, # Resize all images to the value before training
    patience=10, # early stopping
    batch=2, # Fraction means fraction of device memory
    optimizer='Adam',
    single_cls=True, # True when need to find an object, not classsify it
    cos_lr=True, # Cosine learning rate scheduler
    lr0=1e-3, # Initial learning rate
    lrf=1e-5, # Final learning rate to regulate slope of lr scheduler
    weight_decay=1e-4, # L2 regularization
    warmup_epochs=1, # Num of warmup epochs
    box=7.5, # Weight of box loss in total loss function. Emphasises accurate box prediction
    cls=0, # Weight of classification loss in total loss function. Ephasises correct class prediction
    dfl=0,
    pose=0,
    kobj=0,
    val=True, # Enable validation during training
    plots=True, # Generate and save plots of train and val metrics
    amp=True, # Enables Mixed Precision training
    freeze=None, # Freezes specified num of first layers for transfer learning
    device=0,
    workers=4,
    save=True, # Save checkpoints
    project='checkpoints', # Name of a folder to save training outputs
    name=model_name, # Training run name. Creates subdir in project dir
)

In [ ]:
metrics = model.val()

In [ ]:
model.export(format="onnx")